In [14]:
import glob
from os import path, makedirs
from PIL import Image
import rasterio
import numpy as np
from tqdm.notebook import tqdm

In [5]:
root_path = "/mnt/nvm/projects/b2p-siteident"
rgb_tile_path = "data/rgb_tiles/"

In [12]:
for fp in glob.glob(path.join(root_path, rgb_tile_path, "*")):
    if fp.endswith("_jpg"):
        continue
    print(fp)
    if not path.isdir(fp + "_jpg"):
        makedirs(fp + "_jpg")

/mnt/nvm/projects/b2p-siteident/data/rgb_tiles/v1_b2p_rgb_large_590_320
/mnt/nvm/projects/b2p-siteident/data/rgb_tiles/v1_b2p_rgb_large_1150_600
/mnt/nvm/projects/b2p-siteident/data/rgb_tiles/v2_b2p_rgb_large_590_320
/mnt/nvm/projects/b2p-siteident/data/rgb_tiles/v1_b2p_rgb_large_2350_1200
/mnt/nvm/projects/b2p-siteident/data/rgb_tiles/v2_b2p_rgb_large_1150_600
/mnt/nvm/projects/b2p-siteident/data/rgb_tiles/v2_b2p_rgb_large_2350_1200


In [16]:
for fp in glob.glob(path.join(root_path, rgb_tile_path, "*")):
    if fp.endswith("_jpg"):
        continue
    out_fp = fp + "_jpg"
    for tif_fp in tqdm(glob.glob(path.join(fp, "*.tif")), desc=fp.split("/")[-1]):
        out_file = tif_fp.replace(fp, out_fp).replace(".tif", ".jpg")
        if not path.isfile(out_file):
            with rasterio.open(tif_fp) as src:
                arr = np.concatenate((
                    np.expand_dims(src.read(1), -1),
                    np.expand_dims(src.read(2), -1),
                    np.expand_dims(src.read(3), -1),
                ), -1)
                img = Image.fromarray(arr)
                img.save(out_file)

v1_b2p_rgb_large_590_320:   0%|          | 0/3354 [00:00<?, ?it/s]

v1_b2p_rgb_large_1150_600:   0%|          | 0/3354 [00:00<?, ?it/s]

v2_b2p_rgb_large_590_320:   0%|          | 0/3354 [00:00<?, ?it/s]

v1_b2p_rgb_large_2350_1200:   0%|          | 0/3354 [00:00<?, ?it/s]

v2_b2p_rgb_large_1150_600:   0%|          | 0/2874 [00:00<?, ?it/s]

v2_b2p_rgb_large_2350_1200:   0%|          | 0/3354 [00:00<?, ?it/s]